In [1]:
import sys
import numpy as np 
import torch
# import spconv 

In [2]:
import spconv
%load_ext autoreload

%autoreload 2
# from spconv.conv import SparseConcat

In [12]:
sparse_shape = [10, 20,30]
coors1_np=np.array([0,0,0,0,
                    0,5,5,5,
                    1,2,2,3,
                    1,3,3,3,
                    1,3,3,3,
                    1,3,3,3
                  ]).reshape(-1,4)
coors2_np=np.array([0,0,0,0,
                    0,5,5,5,
                    1,4,5,6,
                    1,4,4,4,
                    1,4,4,4,
                    1,4,4,4,
                  ]).reshape(-1,4)
voxel_features1_np = np.array([1,1,
                             2,2,
                               3,3,
                               4,4,
                               5,5,
                               0,0]).reshape(-1,2)
voxel_features2_np = np.array([11,11,
                             22,22,
                               33,33,
                               44,44,
                               55,55,
                               0,0]).reshape(-1,2)
batch_size=2
device='cpu'
dtype=torch.float32

In [13]:
# sparse_shape = torch.from_numpy(sparse_shape_np).to(device=device)
coors1=torch.from_numpy(coors1_np).to(device=device, dtype=dtype)
coors2=torch.from_numpy(coors2_np).to(device=device, dtype=dtype)

voxel_features1 = torch.from_numpy(voxel_features1_np).to(device=device, dtype=dtype).requires_grad_()
voxel_features2 = torch.from_numpy(voxel_features2_np).to(device=device, dtype=dtype).requires_grad_()

In [14]:

coors1 = coors1.int()
coors2 = coors2.int()
tensor1= spconv.SparseConvTensor(voxel_features1, coors1, sparse_shape,
                                      batch_size)
tensor2= spconv.SparseConvTensor(voxel_features2, coors2, sparse_shape,
                                      batch_size)

In [15]:
sp_cat = spconv.SparseConcat3d(indice_key='spcat0')

In [16]:
t = sp_cat(tensor1, tensor2)

In [17]:
t.features

tensor([[ 1.,  1., 11., 11.],
        [ 2.,  2., 22., 22.],
        [ 3.,  3.,  0.,  0.],
        [ 4.,  4.,  0.,  0.],
        [ 5.,  5.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0., 33., 33.],
        [ 0.,  0., 44., 44.],
        [ 0.,  0., 55., 55.],
        [ 0.,  0.,  0.,  0.]], grad_fn=<SparseConcatFunctionBackward>)

In [18]:
tensor1.features

tensor([[1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [0., 0.]], requires_grad=True)

In [19]:
tensor2.features

tensor([[11., 11.],
        [22., 22.],
        [33., 33.],
        [44., 44.],
        [55., 55.],
        [ 0.,  0.]], requires_grad=True)

In [20]:
tensor1.indice_dict['spcat0'][2]

tensor([[[ 0,  1,  2,  3,  4,  5,  6,  6,  6,  6, -1, -1],
         [ 0,  1,  6,  6,  6,  6,  2,  3,  4,  5, -1, -1]]], dtype=torch.int32)

In [21]:
tensor1.indice_dict['spcat0'] # outids, indices1,indice_pairs, indice_pair_num,spatial_shape1

(tensor([[0, 0, 0, 0],
         [0, 5, 5, 5],
         [1, 2, 2, 3],
         [1, 3, 3, 3],
         [1, 3, 3, 3],
         [1, 3, 3, 3],
         [1, 4, 5, 6],
         [1, 4, 4, 4],
         [1, 4, 4, 4],
         [1, 4, 4, 4]], dtype=torch.int32), tensor([[0, 0, 0, 0],
         [0, 5, 5, 5],
         [1, 2, 2, 3],
         [1, 3, 3, 3],
         [1, 3, 3, 3],
         [1, 3, 3, 3]], dtype=torch.int32), tensor([[[ 0,  1,  2,  3,  4,  5,  6,  6,  6,  6, -1, -1],
          [ 0,  1,  6,  6,  6,  6,  2,  3,  4,  5, -1, -1]]], dtype=torch.int32), tensor([10], dtype=torch.int32), [10,
  20,
  30])

In [22]:
g_out = torch.rand_like(t.features)

t.features.backward(g_out)

In [23]:
g_out

tensor([[0.6131, 0.2677, 0.3235, 0.8158],
        [0.9890, 0.6911, 0.3143, 0.8138],
        [0.3281, 0.9017, 0.2168, 0.2618],
        [0.9920, 0.2907, 0.4479, 0.3596],
        [0.6361, 0.4242, 0.5477, 0.1727],
        [0.5500, 0.3042, 0.2333, 0.7106],
        [0.4534, 0.5604, 0.3324, 0.2786],
        [0.6336, 0.9248, 0.1487, 0.2284],
        [0.2468, 0.4768, 0.1731, 0.0926],
        [0.0329, 0.0661, 0.7620, 0.6735]])

In [24]:
tensor1.features.grad

tensor([[0.6131, 0.2677],
        [0.9890, 0.6911],
        [0.3281, 0.9017],
        [0.9920, 0.2907],
        [0.6361, 0.4242],
        [0.5500, 0.3042]])

In [25]:
tensor2.features.grad

tensor([[0.3235, 0.8158],
        [0.3143, 0.8138],
        [0.3324, 0.2786],
        [0.1487, 0.2284],
        [0.1731, 0.0926],
        [0.7620, 0.6735]])